In [2]:
import pandas as pd

In [3]:
df=pd.read_csv("train.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df.describe()

,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [7]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
df=df.fillna('')

In [9]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [10]:
df.columns

Index(['id', 'title', 'author', 'text', 'label'], dtype='object')

In [11]:
df=df.drop(['id', 'title', 'author'], axis=1)

In [12]:
df.head()

,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,Ever get the feeling your life circles the rou...,0
2,"Why the Truth Might Get You Fired October 29, ...",1
3,Videos 15 Civilians Killed In Single US Airstr...,1
4,Print \nAn Iranian woman has been sentenced to...,1


In [13]:
from nltk.corpus import stopwords

In [14]:
from nltk.stem.porter import PorterStemmer

In [15]:
import re

In [16]:
port_stem=PorterStemmer()

In [17]:
port_stem

<PorterStemmer>

In [18]:
port_stem.stem("Hi thIs is chando * % %@@@")

'hi this is chando * % %@@@'

In [19]:
def stemming(content):
    con=re.sub('[^a-zA-Z]', ' ', content)
    con=con.lower()
    con=con.split()
    con=[port_stem.stem(word) for word in con if not word in stopwords.words('english')]
    con=' '.join(con)
    return con

In [20]:
stemming('Hi this is chando')

'hi chando'

In [21]:
df['text']= df['text'].apply(stemming)

In [22]:
x=df['text']

In [23]:
y=df['label']

In [24]:
y.shape

(20800,)

In [61]:
from sklearn.model_selection import train_test_split, GridSearchCV

In [27]:
x_train , x_test , y_train, y_test = train_test_split(x, y, test_size=0.20)

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [29]:
vect=TfidfVectorizer()

In [30]:
x_train=vect.fit_transform(x_train)
x_test=vect.transform(x_test)

In [31]:
x_test.shape

(4160, 99169)

Logistic Regression

In [32]:
from sklearn.linear_model import LogisticRegression

In [33]:
model_lr = LogisticRegression()

In [48]:
model_lr.fit(x_train, y_train)

LogisticRegression()

In [49]:
prediction_lr=model_lr.predict(x_test)
prediction_lr

array([1, 1, 0, ..., 1, 0, 1], dtype=int64)

In [50]:
model_lr.score(x_test, y_test)

0.9420673076923077

In [90]:
pickle.dump(model_lr, open('lr.pkl', 'wb'))

In [ ]:
pickle.dump(model_lr, open('Logistic_Regression.pkl', 'wb'))

Support Vector Machine(SVM)

In [53]:
from sklearn.svm import SVC

In [54]:
svc_model= SVC()

In [55]:
svc_model.fit(x_train, y_train)

SVC()

In [56]:
svc_prediction=svc_model.predict(x_test)

In [57]:
svc_model.score(x_test, y_test)

0.9507211538461539

In [92]:
pickle.dump(svc_model, open('svm.pkl', 'wb'))

Random Forest

In [59]:
from sklearn.ensemble import RandomForestClassifier

In [60]:
clf= RandomForestClassifier(random_state=42)
params={
    "n_estimators": range(50,125,25),
    "max_depth": range(60,81,2)
}
params

{'n_estimators': range(50, 125, 25), 'max_depth': range(60, 81, 2)}

In [62]:
rfc_model = GridSearchCV(
    clf,
    param_grid= params,
    cv= 5,
    n_jobs= -1,
    verbose=1
)

In [63]:
rfc_model.fit(x_train,y_train)  

Fitting 5 folds for each of 33 candidates, totalling 165 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(60, 81, 2),
                         'n_estimators': range(50, 125, 25)},
             verbose=1)

In [64]:
rfc_prediction=rfc_model.predict(x_test)
rfc_prediction

array([1, 1, 0, ..., 0, 0, 1], dtype=int64)

In [65]:
rfc_model.score(x_test, y_test)

0.9122596153846154

In [88]:
pickle.dump(rfc_model, open('rfc.pkl', 'wb'))

Decison Tree Classifier

In [34]:
from sklearn.tree import DecisionTreeClassifier

In [35]:
model=DecisionTreeClassifier()

In [36]:
model.fit(x_train, y_train)

DecisionTreeClassifier()

In [37]:
prediction=model.predict(x_test)

In [38]:
prediction

array([1, 1, 0, ..., 1, 1, 1], dtype=int64)

In [39]:
model.score(x_test, y_test)

0.8786057692307693

In [40]:
import pickle

In [41]:
pickle.dump(vect, open('vector.pkl', 'wb'))

In [86]:
pickle.dump(model, open('dtc.pkl', 'wb'))

In [43]:
vector_form=pickle.load(open('vector.pkl', 'rb'))

In [93]:
load_model=pickle.load(open('svm.pkl', 'rb'))

In [94]:
def fake_news(news):
    news=stemming(news)
    input_data=[news]
    vector_form1=vector_form.transform(input_data)
    prediction = load_model.predict(vector_form1)
    return prediction

In [96]:
val=fake_news("""In these trying times, Jackie Mason is the Voice of Reason. [In this week’s exclusive clip for Breitbart News, Jackie discusses the looming threat of North Korea, and explains how President Donald Trump could win the support of the Hollywood left if the U. S. needs to strike first.  “If he decides to bomb them, the whole country will be behind him, because everybody will realize he had no choice and that was the only thing to do,” Jackie says. “Except the Hollywood left. They’ll get nauseous. ” “[Trump] could win the left over, they’ll fall in love with him in a minute. If he bombed them for a better reason,” Jackie explains. “Like if they have no transgender toilets. ” Jackie also says it’s no surprise that Hollywood celebrities didn’t support Trump’s strike on a Syrian airfield this month. “They were infuriated,” he says. “Because it might only save lives. That doesn’t mean anything to them. If it only saved the environment, or climate change! They’d be the happiest people in the world. ” Still, Jackie says he’s got nothing against Hollywood celebs. They’ve got a tough life in this country. Watch Jackie’s latest clip above.   Follow Daniel Nussbaum on Twitter: @dznussbaum """)

In [97]:
if val==[0]:
    print('reliable')
else:
    print('unreliable')

reliable
